In [1]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [3]:
#making a directory
!mkdir ~/.kaggle

In [ ]:
#Copy the kaggle.json to created folder
!/content/kaggle.json ~/.kaggle/

In [5]:
#permisson for the json to act
! chmod 600 /content/kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d brendan45774/test-file

In [7]:
! unzip /content/test-file.zip

Archive:  /content/test-file.zip
  inflating: tested.csv              


In [8]:
import pandas as pd
df=pd.read_csv('/content/tested.csv')

In [9]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


**Data Cleaning**

In [10]:
df.drop(columns=['PassengerId','Name','Cabin','Ticket'],inplace=True)

In [101]:
# selecing the 2nd row
np.array(df.iloc[1])

array([1, 3, 'female', 47.0, 1, 0, 7.0, 'S'], dtype=object)

In [11]:
df.sample(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
21,0,3,male,9.00,0,1,3.1708,S
76,0,3,male,NaN,0,0,8.0500,S
250,1,2,female,0.92,1,2,27.7500,S
7,0,2,male,26.00,1,1,29.0000,S
414,1,1,female,39.00,0,0,108.9000,C


In [20]:
df.shape

(418, 8)

In [12]:
df.isnull().sum()

Survived     0
Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

**train_test_split is found in model_selection**

In [15]:
from sklearn.model_selection import train_test_split

In [18]:
x_train , x_test , y_train , y_test = train_test_split(df.drop(columns=['Survived']) , df['Survived'] , test_size=0.3 , random_state=0)

# we do fit when we need any preprocessing like StandardScale, MinMaxScale etc

In [22]:
x_train.head(5)  #contains independent columns

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
96,1,female,76.0,1,0,78.8500,S
381,3,male,26.0,0,0,7.8792,Q
89,2,male,2.0,1,1,23.0000,S
233,3,male,NaN,0,0,7.8792,Q
191,1,male,NaN,0,0,26.0000,S


In [23]:
y_train.head(5) #contains dependent/target column

96     1
381    0
89     0
233    0
191    0
Name: Survived, dtype: int64

**Preprocessing**

In [30]:
x_train.isnull().sum()

Pclass       0
Sex          0
Age         56
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

In [33]:
x_train.sample(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
317,2,male,19.0,0,0,10.5000,S
147,3,male,22.0,0,0,8.0500,S
270,1,male,46.0,0,0,75.2417,C
278,2,male,30.0,1,0,21.0000,S
44,1,female,45.0,1,0,52.5542,S


In [41]:
x_train.shape

(292, 7)

In [34]:
x_train.nunique()

Pclass        3
Sex           2
Age          73
SibSp         6
Parch         8
Fare        132
Embarked      3
dtype: int64

here we can analysis that
Sex , Embarked are the Nomical Categorical Dataset.
if Pclass is in categorical dataset then it will Ordinal cat. dataset

In [35]:
from sklearn.preprocessing import OrdinalEncoder , OneHotEncoder , MinMaxScaler , Sca
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

**Column Transformer**

In [107]:
# Imputer Transformer to handle the missing value with the help of SimpleImputer
trf1 = ColumnTransformer( [
    ('impute_age',SimpleImputer(strategy='mean'),[2,5]) ] ,  #we use SimpleImputer because we filling the values
      remainder='passthrough')        #[2,5] , It is best pratice to use the index of the column_name instead of the column names

In [109]:
# OneHotEncoder      sprase , handle_unknown are the parameters of OneHotEncoder
trf2 = ColumnTransformer( [            #here we want to make prediction that's why we do not drop the first column
        ('ohe_sex_embarked', OneHotEncoder( sparse_output = False , handle_unknown='ignore') , [1,6]) ] ,
    remainder='passthrough'
)
#   sparse is bool type ,  default is true means it will return a sparse matrix and false means dense array
#   handle_unknown is str type , default is 'error' means if unknown categories are encountered during transformation.
#      and if handle_unknown is 'ignore' means This ignores unknown categories and returns a result with zeros in their place.

In [110]:
# Scaling                  This slice object can then be used to extract a portion of a sequence, such as a list, tuple, or string.
trf3=ColumnTransformer( [   # ex = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]---> my=slice(0,3)----> lst=[my]-->print(lst)-->[0,1,2]
        ('scale' , MinMaxScaler() , slice(0,8)) ]
)  #here we not use the StandardScaler because we use the DecisionTreeRegression which show no effect on StandardScaler

In [111]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest ,chi2

In [112]:
# feature selection

trf4 = SelectKBest( score_func = chi2 , k=8); # k means only top 8 colums out of 10 are uses.
# Note : before OneHotEncoder ther are 7 columns in x_train . During OneHotEncoder , colums no. 1(sex) is Replace with 2 column
#     and column no. 6(Embarked) is replace with  3 columns , so total column is (7-2+2+3=10) 10. We do drop first column bcz we're using DTC

# The Chi-Square test evaluates the independence of a feature with respect to the target variable.
# It measures how much the observed distribution of the feature differs from the expected distribution
# if the feature and target were independent.

In [113]:
#  Train the Model
trf5=DecisionTreeClassifier()

In [114]:
from sklearn.pipeline import Pipeline , make_pipeline

**Create  Pipeline**

In [115]:
pipe = Pipeline( [
    ( 'trf1' , trf1 ) ,                 # Pipeline take a list in which it takes the value in tuple and
    ( 'trf2' , trf2 ) ,                  # each tuple contains 2 values 1st is transformer_name and 2nd is his object
    ( 'trf3' , trf3 ) ,
    ( 'trf4' , trf4 ) ,
    ( 'trf5' , trf5 )
] )

# alternate way
# pipe=make_pipeline( trf1 , trf2 , trf3 , trf4 , trf5)

'Pipeline' requires the naming of steps and 'make_pipeline' does not

same goes too ColumnTransformer and make_column_transformer

In [116]:
pipe.fit(x_train,y_train)  # training the model

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2, 5])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 8, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7f3dafbb8280>)),
                ('trf5', DecisionTreeClassifier())])

In [117]:
# through pipe.fit(x_train , y_traint ) it trained the model
pipe.named_steps['trf1'].transformers_[0][1].statistics_  # it tell the fill mean value in place of missing numberical data

array([30.48974576, 31.89851168])

In [118]:
# predict
y_pred=pipe.predict(x_test)

In [119]:
from sklearn.metrics import accuracy_score
accuracy_score( y_test , y_pred )

0.5634920634920635

**CrossVAlidation Using Pipeline**

In [120]:
from sklearn.model_selection import cross_val_score
cross_val_score( pipe , x_train , y_train , cv = 5 , scoring='accuracy').mean()

0.667855055523086

**GridSearch using Pipeline** learn in future

**Exporting the Pipeline**

In [92]:
# export
import pickle
pickle.dump( pipe , open( 'pipe.pkl','wb') )

**Procuction Code**

In [121]:
import pickle
import numpy as np

In [123]:
pipe=pickle.load(open('/content/pipe.pkl','rb'))

In [124]:
# assume user input as
input=np.array([3, 'female', 47.0, 1, 0, 7.0, 'S'], dtype=object).reshape(1,7)

In [125]:
pipe.predict(input)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0])